# A toy example for modeling complex drug-protein interactions using RAF kinases and RAF inhibitors

Here provide the step-by-step construction code for a toy example to model complex drug-protein interactions using PySB with energy formulation (through support for eBNG). This example describes RAF kinases as drug targets and RAF inhibitors as the drug. To run this code you'll need to have Pysb with BNG installed, please follow instructions at: http://pysb.org/ .

### Manual definition of the biochemical reaction system

To start, we import all required Pysb classes and instantiate the model:

In [48]:
from pysb import Model, Monomer, Parameter, Expression,  Rule, Observable, Initial, Annotation, EnergyPattern, ANY
from pysb.bng import generate_equations
from pysb.export import export
from sympy import exp, log

Model();

Next, we define the two basic components of the model, RAF kinases (R) and RAF inhibitors (I): 

In [49]:
#define a monomer R that represents a RAF kinase with a binding site for RAF (r) and another for the drug (i)
Monomer('R', ['r', 'i']);   
#define a monomer I that represents a RAF inhibitor with a binding site for RAF (r) 
Monomer('I',['r']);   

We define the parameters for initializing abundance of components:

In [50]:
#define the initial conditions for R and I
Parameter('R_0',1.0);  # uM
Parameter('I_0',1.0);  # uM
Initial(R(r=None, i=None), R_0);
Initial(I(r=None), I_0);

Then, we define the kinetic parameters and thermodynamic factors: 

In [51]:
#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for RAF dimerization
Parameter('RR_kD',1.0);  #uM
Parameter('RR_kf',1.0);  #/s/uM
Parameter('RR_phi',1.0); #unitless 

#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for drug binding to RAF
Parameter('RI_kD',1.0);  #uM
Parameter('RI_kf',1.0);  #/s/uM
Parameter('RI_phi',1.0); #unitless

#define thermodynamic factors f and g
Parameter('f',1.0);  #unitless 
Parameter('g',1.0);  #unitless

We convert the kinetic parameters into corresponding energy parameters:

In [52]:
#convert kinetic parameters into energies for RAF dimerization
Expression('Gf_RR', log(RR_kD)); #unitless 
Expression('Ea0_RR',-RR_phi*log(RR_kD)-log(RR_kf)); #unitless 

#convert kinetic parameters into energies for drug binding to RAF
Expression('Gf_RI', log(RI_kD)); #unitless 
Expression('Ea0_RI',-RI_phi*log(RI_kD)-log(RI_kf)); #unitless 

#convert thermodynamic factors into energies
Expression('Gf_f',log(f)); #unitless 
Expression('Gf_g',log(g)); #unitless 

We define the energy patterns to assign the energy within species:

In [53]:
# define energy in bond between R and R
EnergyPattern('ep_RR',R(r=1)%R(r=1),Gf_RR);

# define energy in bond between R and I
EnergyPattern('ep_RI',R(i=1)%I(r=1),Gf_RI);

# define additional energy in bond betwee RAF dimer and a single drug molecule
Expression('Gf_RRI',Gf_f);
EnergyPattern('ep_RRI',R(r=1,i=None)%R(r=1,i=2)%I(r=2), Gf_RRI);

# define additional energy in bond betwee RAF dimer and two drug molecules
Expression('Gf_IRRI', Gf_f + Gf_g);
EnergyPattern('ep_IRRI',I(r=2)%R(r=1,i=2)%R(r=1,i=3)%I(r=3), Gf_IRRI);

We define observables useful later on to investigate the behaviour of model simulations: 

In [54]:
# define observable for total RAF and total drug
Observable('Rtot_obs', R());
Observable('Itot_obs', I());

# define observable for drug unbound RAF monomer
Observable('R_obs', R(i=None));

# define observable for RAF dimer unbound by drug
Observable('RR_obs', R(r=1,i=None)%R(r=1,i=None));

# define observable for RAF dimer bound by single drug
Observable('RRI_obs', R(r=1,i=None)%R(r=1,i=2)%I(r=2));

# define observable for RAF dimer bound by double drug
Observable('IRRI_obs', I(r=2)%R(r=1,i=2)%R(r=1,i=3)%I(r=3));

As the last step in the model construction, we define the reactions for RAF dimerization and drug binding: 

In [55]:
#define RAF dimerization reaction
Rule('RR', R(r=None)+R(r=None) | R(r=1)%R(r=1) , RR_phi, Ea0_RR, energy=True);

#define drug binding to RAF reaction
Rule('RI', R(i=None)+I(r=None) | R(i=1)%I(r=1) , RI_phi, Ea0_RI, energy=True);

### Automatic generation of kinetic model

We generate the kinetic model by passing the information on the model to BNG, parse the reaction network that is returned and list the properties of the resulting kinetic model: 

In [56]:
# generate the model equations
generate_equations(model)

# print model infomration
print ('RAF-RAFi model information')
print ('Species:',len(model.species))
print ('Parameters:',len(model.parameters)+len(model.initial_conditions))
print ('Expressions:',len(model.expressions))
print ('Observables:', len(model.observables))
ntotr=len(model.rules);
nenergy=len([r for r in model.rules if r.energy]);
print ('Total Rules:', ntotr)
print ('Energy Rules:', nenergy)
print('Non-energy Rules:', ntotr-nenergy)
print('Energy Patterns:', len(model.energypatterns))
print('Reactions:',len(model.reactions))

RAF-RAFi model information
Species: 6
Parameters: 12
Expressions: 20
Observables: 6
Total Rules: 2
Energy Rules: 2
Non-energy Rules: 0
Energy Patterns: 4
Reactions: 12


We plot the automatically generated equation rates to check that thermodynamic factors indeed control cooperative reaction rates:

In [57]:
# to appear

### Model simulations

We use the generated model to simulate the response of RAF kinases to three classes of RAF inhibitors: 1st generation (e.g. Vemurafenib, Dabrafenib and Encorafenib), paradox breakers (e.g. PLX8349) and panRAF (e.g. LY3009120, AZ628) inhibitors. We observe the effect of the drugs in situations with low and high propensity for RAF dimerization (controlled by setting the Kd values of the RAF dimerization reactions) to study the effect that the drugs have in absence or presence of the dimerization status of RAFs, for example as induced by Ras-GTP signal that induces RAF dimerization. 

In [64]:
#import various utility packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import ODE simulator
from pysb.simulator import ScipyOdeSimulator

#set up the ODE simulator for the model
sim = ScipyOdeSimulator(model, atol=1E-50);

#set the dilution range for the RAF inhibitor
RAFi_dil=np.logspace(-4, 1, 11, base=10.0); #uM
#set the values of f and g to be simulated
#1st generation: f= 0.001, g=1000
#paradox breaker: f= 1.0, g=1000
#panRAF: f= 0.001, g=1
ff=[0.001, 1.0, 0.001];
gg=[1000, 1000, 1];

#set the time of simulation
t_start = 0.0;
t_end = 10.0;
tspan = np.linspace(t_start, t_end);

#simulate ODE systems
#sim.run(tspan=tspan)
#res = sim.run(tspan=tspan)